## p-value for model selection

There are some changes we could make to the model selection process, so we will investigate one now.  We do not want to start p-hacking (trying many options until we get good results), so we will stick with our previously selected models.  This investigation is just out of curiosity to see if other methods could be used in future for making better decisions.

We will investigate the use of p-values in deciding whether there is a significant difference between the null hypothesis (baseline model) and the alternative hypothesis (either the Average or Ridge model).  If the difference between model predictions are not statistically significant, then it would make sense to stick with the simpler baseline model.  Often a p-value of 0.05 is chosen as a cut-off to identify statitically significant differences.  Anything lower than this is considered statistically significant.

References:
1. Applying p-value significance testing: https://machinelearningmastery.com/parametric-statistical-significance-tests-in-python/
1. Limitations of p-values: https://sphweb.bumc.bu.edu/otlt/MPH-Modules/EP/EP713_RandomError/EP713_RandomError7.html
1. p-hacking in the larger scientific community: https://fivethirtyeight.com/features/science-isnt-broken/

In [1]:
# common notebook config
%run notebook-config.ipynb

autoreload enabled
numpy imported as np (1.19.5)
pandas imported as pd (1.2.1)
Pandas display: Remove maximum column width
Pandas display: Show up to 100 columns in tables
Pandas display: Show up to 100 rows in tables
Pandas display: Set floats to show up to 3 decimal places
matplotlib: show plots inline
matplotlib imported as mpl (3.3.3)
matplotlib.pyplot imported as plt
matplotlib: use ggplot style
seaborn: set white grid theme
Logging: show log messages in ipython


In [2]:
# we use ttest_rel because we will be comparing predictions made on related samples (cross-validation folds)
#  rather than independent samples (which would require the use of ttest_ind)
from scipy.stats import ttest_rel

import warnings
warnings.filterwarnings('ignore')

models_filepath = "../models"

In [3]:
models_df = pd.read_csv("{}/championship_transformation_selected_models.csv".format(models_filepath))

# Calculate root mean squared error which has the same units as the target variables
models_df["rmse"] = np.sqrt(models_df["mse"])

In [4]:
def calc_p_values(df):
    """ Calculate p-values for each target variable and season
    
    Return pivot table showing test RMSE and p-value per target variable and season,
    and which model was selected as the best by taking the lowest average previous RMSE
    """
    # Make sure we are just using the tuned models
    df = df[(df.tuned_model)]

    # Set order of models (so they appear in this order in the pivot table later)
    ordered_model_names = ["Baseline", "Average", "Ridge"]
    df['model_name'] = pd.Categorical(
            df['model_name'],
            ordered_model_names,
            ordered=True)

    # Preprocessing to gather prior model perforances together for easier access later
    def list_of_previous_values(x):
        return [list(val.values) for val in x.shift(1).expanding()]

    df["previous_rmses"] = df.groupby(["model_name", "target_variable"])["rmse"].transform(list_of_previous_values)

    # Pivot the tuned models so that there is one row per target_variable and season combination
    pivot_df = df.pivot(index=["target_variable", "season"],
                        columns='model_name',
                        values=["best_model", "rmse", "previous_rmses"])

    # Calculate p-value using ttest between the baseline RMSEs and each model RMSEs
    def calc_p_value_with_baseline(x):
        p_values = []
        for model_name in ordered_model_names:
            # TODO: would be interesting to investigate the impact of the "alternative" parameter
            #  e.g. with a value of "less" or "greater" rather than "two-sided"
            #  but this doesn't accept NaN values
            tstat, p_value_with_baseline = ttest_rel(x.Baseline, x[model_name], nan_policy="omit")
            p_values.append(p_value_with_baseline)
        return p_values

    p_values = pivot_df["previous_rmses"].apply(calc_p_value_with_baseline, axis=1, result_type="broadcast")

    # Save p-values as new columns to the pivot table
    pivot_df["p_value_with_baseline", "Baseline"] = p_values.Baseline
    pivot_df["p_value_with_baseline", "Average"] = p_values.Average
    pivot_df["p_value_with_baseline", "Ridge"] = p_values.Ridge


    return pivot_df[["best_model", "rmse", "p_value_with_baseline"]]

We can start by looking at the p-values for home_corners_against_mean_premier. The previous model selection performed poorly for this value and chose the ridge model for many seasons when the baseline performed better.

In [5]:
filtered_df = models_df[(models_df.target_variable == "home_corners_against_mean_premier")]

calc_p_values(filtered_df)

best_model                    rmse  \
model_name                                 Baseline Average  Ridge Baseline   
target_variable                   season                                      
home_corners_against_mean_premier 2002         True   False  False    0.928   
                                  2003         True   False  False    1.179   
                                  2004        False    True  False    0.582   
                                  2005        False   False   True    1.024   
                                  2006        False   False   True     0.72   
                                  2007        False   False   True    0.258   
                                  2008        False   False   True    0.401   
                                  2009        False   False   True    0.575   
                                  2010        False   False   True     0.96   
                                  2011        False   False   True    0.779   
                                  2012        False   False   True    0.269   
                                  2013        False   False   True    0.876   
                                  2014        False   False   True    0.585   
                                  2015        False   False   True    0.803   
                                  2016         True   False  False     0.71   
                                  2017         True   False  False    0.492   
                                  2018         True   False  False    0.602   
                                  2019        False   False   True    1.075   
                                  2020        False   False   True    0.731   

                                                       p_value_with_baseline  \
model_name                               Average Ridge              Baseline   
target_variable                   season                                       
home_corners_against_mean_premier 2002     0.928 0.928                    --   
                                  2003     0.907 0.907                    --   
                                  2004     0.481 0.478                    --   
                                  2005     1.213 1.136                    --   
                                  2006     0.784 0.784                    --   
                                  2007     0.314 0.122                    --   
                                  2008     0.448 0.434                    --   
                                  2009      0.55 0.564                    --   
                                  2010     0.966 0.913                    --   
                                  2011     0.637 0.613                    --   
                                  2012     0.612 0.724                    --   
                                  2013     0.859 0.898                    --   
                                  2014     0.595 0.632                    --   
                                  2015     0.983 0.965                    --   
                                  2016     0.572 0.602                    --   
                                  2017     0.648 0.539                    --   
                                  2018     0.601 0.536                    --   
                                  2019     1.021 1.073                    --   
                                  2020     0.695 0.634                    --   

                                                        
model_name                               Average Ridge  
target_variable                   season                
home_corners_against_mean_premier 2002        --    --  
                                  2003        --    --  
                                  2004       0.5   0.5  
                                  2005     0.258 0.254  
                                  2006     0.668  0.48  
                                  2007     0.776 0.591  
                            

Note that '--' indicates the values used in calculating the p-value were identical.  The baseline column is all '--' because it is being compared with itself.

All of the p-values in this case are considerably above 0.05 indicating there is no significant performance difference between the baseline and either the average or ridge models.  Using this method we would have always chosen to stick with the baseline model for this target variable.  In hindsight this would have resulted in better model choices.

Now let's look at the p-values for home_shotsOnTarget_for_mean_premier.  In this case the previous model_selection method did well to choose the ridge model over the baseline.

In [6]:
filtered_df = models_df[(models_df.target_variable == "home_shotsOnTarget_for_mean_premier")]

calc_p_values(filtered_df)

best_model                    rmse  \
model_name                                   Baseline Average  Ridge Baseline   
target_variable                     season                                      
home_shotsOnTarget_for_mean_premier 2002         True   False  False    1.583   
                                    2003        False   False   True    1.063   
                                    2004         True   False  False    0.802   
                                    2005         True   False  False     0.41   
                                    2006         True   False  False    0.625   
                                    2007         True   False  False    0.342   
                                    2008         True   False  False      1.3   
                                    2009        False   False   True    0.498   
                                    2010        False   False   True     2.21   
                                    2011        False   False   True    0.997   
                                    2012        False   False   True    1.544   
                                    2013        False   False   True     2.26   
                                    2014        False   False   True    1.788   
                                    2015        False   False   True    1.953   
                                    2016        False   False   True    2.386   
                                    2017        False   False   True    2.177   
                                    2018        False   False   True     1.35   
                                    2019        False   False   True    1.504   
                                    2020        False   False   True    1.773   

                                                          \
model_name                                 Average Ridge   
target_variable                     season                 
home_shotsOnTarget_for_mean_premier 2002     1.583 1.582   
                                    2003     1.063 1.066   
                                    2004     0.962  0.94   
                                    2005      0.41 0.292   
                                    2006     0.628 0.694   
                                    2007     0.342 0.355   
                                    2008       1.3  0.87   
                                    2009     0.498 0.436   
                                    2010     2.201 1.713   
                                    2011     0.402 0.495   
                                    2012     1.487 0.952   
                                    2013     2.936 2.939   
                                    2014     1.788 0.969   
                                    2015     0.914 1.448   
                                    2016     1.029 0.572   
                                    2017     0.575 0.472   
                                    2018     0.685   1.0   
                                    2019     0.221 0.284   
                                    2020     1.087 1.022   

                                           p_value_with_baseline                
model_name                                              Baseline Average Ridge  
target_variable                     season                                      
home_shotsOnTarget_for_mean_premier 2002                      --      --    --  
                                    2003                      --      --    --  
                                    2004                      --      --  0.56  
                                    2005                      --   0.423 0.411  
                                    2006                      --   0.391  0.92  
                                    2007                      --   0.362 0.688  
                                    2008                      --   0.352 0.639  
                                    2009                      --   0.345 0.532  
                                    2010        

Using this method, the model selection would have stuck with the baseline until 2018 at which point the Ridge model was identified to be significantly different with a p-value of 0.032.

The second reference link above points out that p-values depend on sample size - leading to lower p-values for more samples and larger p-values for fewer samples.  This implies that the null-hypothesis is more likely to be selected with fewer samples.  This could explain to an extent why the ridge model was not chosen as statistically significant until there were quite a few earlier seasons.

## Summary

* The p-value is useful for determining if the difference in performance of two models is statistically significant. This can be used in deciding whether to stick with a simpler model or choose a more complex one which appears to perform better.
* Care should be taken to plan the decision metrics before carrying out any tests, otherwise it is possible to fall in to the trap of p-hacking.  This can result in overly optimistic performance estimates.
* It is perhaps more useful when there are more samples to carry out the t-test on.  We can see that it is less likely to reject the null hypothesis with very few samples.
* Another option would be to use standard deviations of mean model performances. We would only accept the more complex models if they perform better than a pre-specified number of standard deviations away from the baseline model performance.
* Incorporating this in to our previous model selection process would have prevented the acceptance of the more complex models (Average and Ridge) until they showed a significant improvement over the baseline.